<h1>Рекуррентная нейронная сеть</h1>

Рекуррентная нейронная сеть (RNN) - тип нейронной сети, где основной идеей является постепенное распространение информации. Используется для обработки временных рядов и текста. Так раз идеально подходит для нашей ситуации.

Для реализации такой нейросети я буду использовать Keras и вспомогающие функции и классы sklearn.

In [1]:
import os

import pandas as pd
import numpy as np

# layers for nntw
from tensorflow.keras.layers import Flatten, Dense, LSTM
from tensorflow.keras.models import Sequential

from tensorflow.keras.models import load_model

# preprocessing and metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.externals import joblib

C:\ProgramData\Miniconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Также вернём функцию-генератор для загрузки данных и получим тестовые и тренировочные данные.

In [2]:
def gen_data(path:str):
    '''Function, that generates data from path\control and path\patient.'''
    
    # generating control objects
    for file in os.listdir(os.path.join(path, 'control')):
    
        # building full-name of file
        full_file = os.path.join(path, os.path.join('control', file))
        
        # reading .csv file
        df = pd.read_csv(full_file)
        
        # generating activity of object and label
        yield df['activity'].values, 0
        
    
    # generating patient objects
    for file in os.listdir(os.path.join(path, 'patient')):
    
        # building full-name of file
        full_file = os.path.join(path, os.path.join('patient', file))
        
        # reading .csv file
        df = pd.read_csv(full_file)
        
        # generating activity of object and label
        yield df['activity'].values, 1

In [3]:
data = [pair for pair in gen_data('data')]

# X set with values of acitivity
X = np.array([pair[0] for pair in data])

# creating normalizer
normalizer = Normalizer()
X = normalizer.fit_transform(X)

# reshape X for LSTm
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

# Y set with labels
Y = np.array([pair[1] for pair in data])

# we don't need this list 
del data

# splitting data to train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True, test_size=0.2)

print(X_train.shape)

len(X_train), len(X_test)

(29, 1, 20160)


(29, 8)

In [4]:
def build():
    '''Function, that builds RNN with Keras.'''
    model = Sequential()
    
    # first LSTM layer
    model.add(LSTM(64, return_sequences=True))
    
    model.add(LSTM(32))
    model.add(Dense(16, activation='relu'))
    
    # last output layer (sigmoid for binary classification)
    model.add(Dense(1, activation='sigmoid'))
    
    # compile model for binary classification
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
# creating model 
model = build()

# fitting model
model.fit(X_train, Y_train, epochs=25, validation_data=[X_test, Y_test])

# predictions for test data
test_predicted = np.reshape(model.predict(X_test), X_test.shape[0])
test_predicted = np.array([1 if value > 0.5 else 0 for value in test_predicted])

# predictions for train data
train_predicted = np.reshape(model.predict(X_train), X_train.shape[0])
train_predicted = np.array([1 if value > 0.5 else 0 for value in train_predicted])

# output
print(f'Test accuracy score is {accuracy_score(test_predicted, Y_test)}')
print(f'Train accuracy score is {accuracy_score(train_predicted, Y_train)}')

Instructions for updating:
Colocations handled automatically by placer.
Train on 29 samples, validate on 8 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/25
29/29 [==============================] - 3s 112ms/sample - loss: 0.6931 - acc: 0.6897 - val_loss: 0.6888 - val_acc: 0.6250
Epoch 2/25
29/29 [==============================] - 0s 7ms/sample - loss: 0.6884 - acc: 0.5172 - val_loss: 0.6946 - val_acc: 0.3750
Epoch 3/25
29/29 [==============================] - 0s 7ms/sample - loss: 0.6835 - acc: 0.6207 - val_loss: 0.6851 - val_acc: 0.6250
Epoch 4/25
29/29 [==============================] - 0s 6ms/sample - loss: 0.6723 - acc: 1.0000 - val_loss: 0.6829 - val_acc: 1.0000
Epoch 5/25
29/29 [==============================] - 0s 6ms/sample - loss: 0.6559 - acc: 1.0000 - val_loss: 0.6758 - val_acc: 1.0000
Epoch 6/25
29/29 [==============================] - 0s 7ms/sample - loss: 0.6353 - acc: 1.0000 - val_loss: 0.6799 - val_acc: 0.3750
Epoch 7/25
29/29 [=========================

Хоть модель и предсказывает правильно 100% тестовых данных (что тоже не является полным показателем успешности модели), однако нам гораздо важнее вероятность того, что человек болен. Кстати, именно поэтому я не использовал f1-score в качестве метрики оценивания. 

Теперь дообучим модель на тестовых данных и сохраним нормализатор и модель в файл для их дальнейшего использования.

In [6]:
# partial fitting on test data
model.train_on_batch(X_test, Y_test)

# output
print(f'Test accuracy score is {accuracy_score(test_predicted, Y_test)}')
print(f'Train accuracy score is {accuracy_score(train_predicted, Y_train)}')

# saving keras model to .h5 file, using h5 module
model.save('model.h5')

# saving normalizer using joblib
joblib.dump(normalizer, 'normalizer.saved')

Test accuracy score is 1.0
Train accuracy score is 1.0


['normalizer.saved']

Далее мы будем использовать эту модель глубокого обучения и нормализатор в рабочем скрипте predict.py.